In [1]:
# Dependencies
import os
import sys

from sqlalchemy import Column, ForeignKeyConstraint, ForeignKey, Date, DateTime, Integer, String, Boolean
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship, sessionmaker
from sqlalchemy import create_engine

import datetime as dt
from datetime import date,datetime, timedelta

import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.image as mpimg


In [2]:
# Setup Inheritence
Base = declarative_base()

In [3]:
# Point to DB location
engine = create_engine('sqlite:///gradebook.db')

In [4]:
# Define Tables

# No table needed for storing authentication - authentication expires periodically

# Add cohort id??? so i can add multiple classes to one DB??
class Cohort(Base):
    __bind_key__ = "gradebook"
    __tablename__ = "cohort"
    cohort_id = Column(Integer, primary_key=True)
    cohort_name = Column(String(25))
    location = Column(String(50))
    start_date = Column(Date)
    end_date = Column(Date)
    #current_unit or units_complete                      # is this needed???

    
class Person(Base):
    __bind_key__ = "gradebook"
    __tablename__ = "person"
    person_id = Column(Integer, primary_key=True)
    name = Column(String(50))
    role = Column(String(50))                     # TA, SSM, Instructor, Student

    
class Email(Base):
    __bind_key__ = "gradebook"
    __tablename__ = "email"
    email_id = Column(Integer, primary_key=True)
    person_id = Column(Integer, ForeignKey("person.person_id"))
    email = Column(String(50))
    date_added = Column(Date)

    
class Github(Base):              #merge github with person?
    __bind_key__ = "gradebook"
    __tablename__ = "github"
    # github_id = Column(Integer, primary_key=True)
    person_id = Column(Integer, ForeignKey('person.person_id'), primary_key=True)           # foreign key
    github_username = Column(String(50))

    
class Student(Base):
    __bind_key__ = "gradebook"
    __tablename__ = "student"
    student_id = Column(Integer, primary_key=True)
    person_id = Column(Integer, ForeignKey('person.person_id'), primary_key=True)
    cohort_id = Column(Integer, ForeignKey('cohort.cohort_id'), primary_key=True)


# -----------------------------------------------------------------

class Unit(Base):
    __bind_key__ = "gradebook"
    __tablename__ = "unit"
    unit_id = Column(Integer, primary_key=True)
    cohort_id = Column(Integer, ForeignKey('cohort.cohort_id'), primary_key=True)
    unit_number = Column(Integer)
    unit_name = Column(String(50))
    unit_start = Column(Date)
    unit_due = Column(Date)
    hw_submissions = Column(Integer)
    students_enrolled = Column(Integer)
    unit_required = Column(Boolean)
    context_code = Column(String)
    career_assignment_bool = Column(Boolean)
    graded_assignments = Column(String)


    
# from session_details api  
# do I connect this to the unit_id
class Session(Base):
    __bind_key__ = "gradebook"
    __tablename__ = 'session'
    session_id = Column(Integer, primary_key=True)
    cohort_id = Column(Integer, ForeignKey('cohort.cohort_id'), primary_key=True) 
    session_name = Column(String(25))
    unit_id = Column(Integer)     
    session_number = Column(Integer)
    session_chapter = Column(String(10))
    session_start = Column(Date)
    zoom_url = Column(String(50))
    

# ------------------------------------------------------------------------

# how to set this up so I can have a list of updated questions    
class Survey(Base):
    __bind_key__ = "gradebook"
    __tablename__ = "survey"
    question_id = Column(Integer, primary_key=True)
    question_text = Column(String(200))
    date_added = Column(Date)

    
class SurveySet(Base):
    __bind_key__ = "gradebook"
    __tablename__ = "surveyset"
    feedback_id = Column(Integer, primary_key=True)             # foreign key
    question_id = Column(Integer, ForeignKey('survey.question_id'), primary_key=True)             # foreign key
    question_number = Column(Integer)
    

class Week(Base):
    __bind_key__ = "gradebook"
    __tablename__="week"
    week_id = Column(Integer, primary_key=True)
    cohort_id = Column(Integer, ForeignKey('cohort.cohort_id'), primary_key=True)
    week_end_date = Column(Date)
    current_unit = Column(Integer, ForeignKey('unit.unit_id'))
    previous_session = Column(Integer, ForeignKey('session.session_id'))


# Maybe add student_name to this table  
# why is this by email address and not student name or ID
class Feedback(Base):
    __bind_key__ = "gradebook"
    __tablename__ = 'feedback'
    # Here we define columns for the table person
    # Notice that each column is also a normal Python instance attribute.
    feedback_id = Column(Integer, ForeignKey('surveyset.feedback_id'))
    student_id = Column(Integer, ForeignKey('student.student_id'), primary_key=True)             # foreign key     
    week = Column(Integer, ForeignKey('week.week_id'), primary_key=True)                   # is this the same as unit?
    submission_date = Column(Date)
    overall_satisfaction = Column(Integer)
    pace = Column(Integer)
    academic_support = Column(Integer)
    outside_class_productivity = Column(Integer)
    instructor_engagement = Column(Integer)
    instructor_clarity = Column(Integer)
    instructor_knowledge = Column(Integer)
    homework_feedback = Column(Integer)
    outside_class_time_spent = Column(Integer)
    class_comments = Column(String(250))
    instructional_support_comments = Column(String(250))
    

    
# from attendance api, no sessions, one table
# maybe make compound key from session and name
class Attendance(Base):
    __bind_key__ = "gradebook"
    __tablename__ = "attendance"
    session_id = Column(Integer, ForeignKey('session.session_id'), primary_key=True)
    student_id = Column(Integer, ForeignKey('student.student_id'), primary_key=True)
    present = Column(Boolean)
    pending = Column(Boolean)

    
# from grade api
# no sessions, one table
# unit_name could come from unit table and this be a foreign key
class Grade(Base):
    __bind_key__ = "gradebook"
    __tablename__ = 'grade'
#    grade_id = Column(Integer, primary_key=True)                           # probably not needed
    unit_id = Column(Integer, ForeignKey('unit.unit_id'), primary_key=True)
    student_id = Column(Integer, ForeignKey('student.student_id'), primary_key=True)
    hw_submitted = Column(Boolean)
    hw_grade = Column(String(10))
    

# from session details api
# why is this by student ID an not name????
class Arrival(Base):
    __bind_key__ = "gradebook"
    __tablename__ = "arrival"
    student_id = Column(Integer, ForeignKey('student.student_id'), primary_key=True)
    session_id = Column(Integer, ForeignKey('session.session_id'), primary_key=True)
    arrival_time = Column(Integer)

    
class Submissions(Base):
    __bind_key__ = "gradebook"
    __tablename__ = "submission"
    unit_id = Column(Integer, ForeignKey('unit.unit_id'), primary_key=True)
    student_id = Column(Integer, ForeignKey('student.student_id'), primary_key=True)
    first_name = Column(String)
    last_name = Column(String)
    # email = Column(String)
    submission_status = Column(Boolean)
    submission_date = Column(Date)
    submission_notes = Column(String)
    grade = Column(String)
    feedback = Column(String)
    plagiarism = Column(Boolean)


class AttendanceStatus(Base):
    __bind_key__ = "gradebook"
    __tablename__ = 'attendance_status'
    bin = Column(Integer, primary_key=True)
    label = Column(String)

In [5]:
# Create connection and/or database if it does not exist
Base.metadata.create_all(engine)
Base.metadata.bind = engine

In [6]:
# Define session and create session instance
# In this case, preparing to add db schema
DBSession = sessionmaker(bind=engine)
session = DBSession()

In [7]:
# Actually add the schema to the db
session.commit()